# 01 · Data Loading & UPRO/SPXU Simulation

Load SPX 1-minute data and simulate UPRO/SPXU leveraged ETF behavior for the duel strategy.

**Workflow**
- Load SPX/USD 1-minute data from HistData.com
- Filter to regular hours only (09:30-16:00 ET)
- Simulate UPRO (3x long) and SPXU (-3x short) from SPX data
- Save processed data for backtest

In [ ]:
# Add parent directory to path for module imports
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import pandas as pd
from config import get_config, MARKET_OPEN, MARKET_CLOSE
from momentum_lib import load_spx_data, simulate_leveraged_etfs, filter_regular_hours
from histdata_loader import ensure_histdata_prices

config = get_config()
data_dir = Path("../data")
data_dir.mkdir(exist_ok=True)

print("Environment loaded.")

In [ ]:
print("=" * 60)
print("STEP 1: LOADING SPX DATA")
print("=" * 60)

# Download/load SPX data from HistData.com
# Note: SPXUSD data starts from 2019
# Files are saved as prices_YYYY.csv (one per year)
spx_prices = ensure_histdata_prices(
    output_dir=data_dir,  # Changed from output_path to output_dir
    start_year=config.start_year,
    end_year=config.end_year,
    rebuild=False,  # Set to True to force re-download
    verbose=True,
)

print(f"\n[OK] SPX data loaded")
print(f"     Range: {spx_prices.index.min()} to {spx_prices.index.max()}")
print(f"     Total rows: {len(spx_prices):,}")
print("=" * 60)

In [ ]:
print("\n" + "=" * 60)
print("STEP 2: FILTER TO REGULAR HOURS (09:30-16:00 ET)")
print("=" * 60)

# Keep only regular market hours
spx_regular = filter_regular_hours(spx_prices, MARKET_OPEN, MARKET_CLOSE)

print(f"\n[OK] Filtered to regular hours")
print(f"     Before: {len(spx_prices):,} rows")
print(f"     After:  {len(spx_regular):,} rows")
print("=" * 60)

# Preview
spx_regular.head(10)

In [ ]:
print("\n" + "=" * 60)
print("STEP 3: SIMULATE UPRO/SPXU FROM SPX")
print("=" * 60)

# Simulate leveraged ETFs
# UPRO = 3x long SPX
# SPXU = -3x short SPX
upro_df, spxu_df = simulate_leveraged_etfs(spx_regular, config)

print(f"\n[OK] Simulated leveraged ETFs")
print(f"     UPRO leverage: {config.upro_leverage}x")
print(f"     SPXU leverage: {config.spxu_leverage}x")
print(f"     Rows: {len(upro_df):,}")
print("=" * 60)

# Show sample
combined_sample = pd.DataFrame({
    "SPX": spx_regular["close"],
    "UPRO": upro_df["close"],
    "SPXU": spxu_df["close"]
})
combined_sample.head(10)

In [ ]:
print("\n" + "=" * 60)
print("STEP 4: SAVE PROCESSED DATA")
print("=" * 60)

# Save simulated UPRO/SPXU data
upro_file = data_dir / "upro_prices.csv"
spxu_file = data_dir / "spxu_prices.csv"

upro_df.to_csv(upro_file, index_label="timestamp")
spxu_df.to_csv(spxu_file, index_label="timestamp")

print(f"\n[OK] Data saved:")
print(f"     UPRO: {upro_file}")
print(f"     SPXU: {spxu_file}")
print("=" * 60)